In [3]:
import struct
import time
from os import path

datfile = '/home/yuzhuyu/u55c/Network_CPU/CPP/processed_data.bin'
# Assume we want to read rows of data with 48 32-bit little endian integers
file_size = path.getsize(datfile)
print(f"The binary dataset has the size of {file_size}")
num_integers = 48
integer_size = 4  # Size of one integer in bytes (4 bytes for 32-bit integer)
row_size = num_integers * integer_size  # Total size of a row in bytes
total_count = file_size // row_size
print(f"The binary dataset contains {total_count} rows.")

days = 8
total_per_file = []
total_per_file.append(total_count)
# reset total per file due to split
num_data_per_split, extras = divmod(total_count, days)
total_per_file = [num_data_per_split] * days
for j in range(extras):
    total_per_file[j] += 1
# split into days (simplifies code later on)
# file_id = 0


sif_output = [[] for _ in range(days)]
# current_file_data = sif_output[file_id]

t0 = time.perf_counter()
if path.exists(datfile):
    with open(datfile, 'rb') as f:  # Open in binary mode
        # while True:
        file_id = 0
        while file_id < days: 

            boundary = total_per_file[file_id]
            read_size = boundary * row_size
            integer_size = boundary * num_integers
            current_file_data = sif_output[file_id]
            
            binary_row = f.read(read_size)  # Read one row of binary data
            if binary_row:  # If there's data to read

                format_string = f'<{integer_size}I'  # Little-endian, 32-bit unsigned integers
                line = struct.unpack(format_string, binary_row)
                
                # if j == boundary:
                
                # MODIFIED: Switch to the next inner list for the new "file"or split of data.
                
                current_file_data.append(line)
                # MODIFIED: Append line to the in-memory data structurinstead of writing to a file.
                file_id += 1
                # j += 1
                # print(integers)
            else:
                print("No data to read or file is empty")
                break
            print("file_id: ", file_id)
else:
    print("No data file")

t1 = time.perf_counter()
print("Total execution time: ", (t1-t0))


The binary dataset has the size of 8801398464
The binary dataset contains 45840617 rows.
file_id:  1
file_id:  2
file_id:  3
file_id:  4
file_id:  5
file_id:  6
file_id:  7
file_id:  8
Total execution time:  104.91418614100257


In [ ]:

# Python program to explain os.read() method  
    
# importing os module  
import os 

path = "/home/yuzhuyu/u55c/Network_CPU/CPP/processed_data_512.bin"
fd = os.open(path, os.O_RDONLY) 
n = 50
readBytes = os.read(fd, n) 
  
# Print the bytes read 
print(readBytes) 
  
# close the file descriptor 
os.close(fd) 

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import sys
from multiprocessing import Manager, Process

import os
from os import path

# import io
# from io import StringIO
# import collections as coll

import numpy as np
import time

def getCriteoAdData(
    datafile,
    o_filename,
    max_ind_range=-1,
    sub_sample_rate=0.0,
    days=7,
    data_split="train",
    randomize="total",
    criteo_kaggle=True,
    memory_map=False,
    dataset_multiprocessing=False,
):

    t0 = time.perf_counter()
    # split the datafile into path and filename
    lstr = datafile.split("/")
    d_path = "/".join(lstr[0:-1]) + "/"
    d_file = lstr[-1].split(".")[0] if criteo_kaggle else lstr[-1]
    npzfile = d_path + ((d_file + "_day") if criteo_kaggle else d_file)
    trafile = d_path + ((d_file + "_fea") if criteo_kaggle else "fea")

    # count number of datapoints in training set
    total_file = d_path + d_file + "_day_count.npz"

    total_count = 0
    total_per_file = []
    if criteo_kaggle:
        if path.exists(datafile):
            t0_0 = time.perf_counter()
            print("Reading data from path=%s" % (datafile))
            with open(str(datafile)) as f:
                for _ in f:
                    total_count += 1
            total_per_file.append(total_count)
            # reset total per file due to split
            num_data_per_split, extras = divmod(total_count, days)
            total_per_file = [num_data_per_split] * days
            for j in range(extras):
                total_per_file[j] += 1
            # split into days (simplifies code later on)
            file_id = 0
            boundary = total_per_file[file_id]
            t0_1 = time.perf_counter()

            print("Counting row number: %s s", (t0_1-t0_0))

            FILE_SIZE = 11147184845
            BLOCK_SIZE = FILE_SIZE
            buffer_count = 0
            fd = os.open(datafile, os.O_RDONLY | os.O_DIRECT)
            try:
                while True:
                    try:
                        buffer = os.read(fd, BLOCK_SIZE)
                        if not buffer:  # If the read returns an empty buffer, EOF is reached
                            break
                        buffer_count += buffer.count(b'\n')
                    except OSError as e:
                        if e.errno == os.errno.EINVAL:
                            # Handle the case where buffer alignment is causing EINVAL
                            pass
                        
            finally:
                os.close(fd)

            t0_2 = time.perf_counter()

            
            print("Reading files: %s s", (t0_2-t0_1))
            print("Reading throughput: ", (FILE_SIZE / (t0_2-t0_1)))
            print("Total bytes: ", buffer_count)

            # #! MODIFIED: Instead of opening new files, initialize a list to store all file data in memory.
            # sif_output = [[] for _ in range(days)]
            # current_file_data = sif_output[file_id]
            # with open(str(datafile)) as f:
            #     for j, line in enumerate(f):
            #         if j == boundary:
            #             file_id += 1
            #             # MODIFIED: Switch to the next inner list for the new "file" or split of data.
            #             current_file_data = sif_output[file_id]
            #             boundary += total_per_file[file_id]
            #         # MODIFIED: Append line to the in-memory data structure instead of writing to a file.
            #         current_file_data.append(line)
            # #! MODIFICATION FINISHES
        else:
            sys.exit(
                "ERROR: Criteo Kaggle Display Ad Challenge Dataset path is invalid; please download from https://labs.criteocom/2014/02/kaggle-display-advertising-challenge-dataset"
            )


    

    t1 = time.perf_counter()
    
    print("Splitting Input Files: %s s", (t1-t0))
    # print("Total Execution Time: %s s", (t5-t0))

    return 0


def loadDataset(
    dataset,
    max_ind_range,
    sub_sample_rate,
    randomize,
    data_split,
    raw_path="",
    pro_data="",
    memory_map=False,
    dataset_multiprocessing=False
):
    # dataset
    if dataset == "kaggle":
        days = 8
        o_filename = "kaggleAdDisplayChallenge_processed"
    elif dataset == "terabyte":
        days = 24
        o_filename = "terabyte_processed"
    else:
        raise (ValueError("Data set option is not supported"))

    # split the datafile into path and filename
    lstr = raw_path.split("/")
    d_path = "/".join(lstr[0:-1]) + "/"
    d_file = lstr[-1].split(".")[0] if dataset == "kaggle" else lstr[-1]
    npzfile = (d_file + "_day") if dataset == "kaggle" else d_file
    # trafile = d_path + ((d_file + "_fea") if dataset == "kaggle" else "fea")

    # check if pre-processed data is available
    data_ready = True
    if memory_map:
        for i in range(days):
            reo_data = d_path + npzfile + "_{0}_reordered.npz".format(i)
            if not path.exists(str(reo_data)):
                data_ready = False
    else:
        if not path.exists(str(pro_data)):
            data_ready = False

    # pre-process data if needed
    # WARNNING: when memory mapping is used we get a collection of files
    if data_ready:
        print("Reading pre-processed data=%s" % (str(pro_data)))
        file = str(pro_data)
    else:
        print("Reading raw data=%s" % (str(raw_path)))
        file = getCriteoAdData(
            raw_path,
            o_filename,
            max_ind_range,
            sub_sample_rate,
            days,
            data_split,
            randomize,
            dataset == "kaggle",
            memory_map,
            dataset_multiprocessing
        )

    return file, days


if __name__ == "__main__":
    ### import packages ###
    import argparse

    ### parse arguments ###
    parser = argparse.ArgumentParser(description="Preprocess Criteo dataset")
    # model related parameters
    parser.add_argument("--max-ind-range", type=int, default=-1)
    parser.add_argument("--data-sub-sample-rate", type=float, default=0.0)  # in [0, 1]
    parser.add_argument("--data-randomize", type=str, default="total")  # or day or none
    parser.add_argument("--memory-map", action="store_true", default=False)
    parser.add_argument("--data-set", type=str, default="kaggle")  # or terabyte
    parser.add_argument("--raw-data-file", type=str, default="")
    parser.add_argument("--processed-data-file", type=str, default="")
    parser.add_argument("--dataset-multiprocessing", action="store_true", default=False)
    args = parser.parse_args()

    loadDataset(
        args.data_set,
        args.max_ind_range,
        args.data_sub_sample_rate,
        args.data_randomize,
        "train",
        args.raw_data_file,
        args.processed_data_file,
        args.memory_map,
        args.dataset_multiprocessing
    )
